In [1]:
# Class: COEN 140 Lab Machine Learning/Data Mining
# Name: Matthew Davenport
# Date: 9/25/2022
# Title: Lab 4 – Iris classifier LDA and QDA without python libraries
# Description: This program will perform clssifications
#           on iris datasets based on the sepal
#           lengths/widths and petal lengths/widths and 
#           classify them as Setosa, Versicolour, or Virginica
#           without using python ML libraries

import numpy as np
import math
import time

ROWS = 150
COLUMNS = 5
CLASSIFICATIONS = 3
FEATURES = ['sepal length','sepal width', 'pedal length', 'pedal width']

In [2]:
# for testing of proper reading of data (provided)
def test_dataset(data):
    if len(data) != 150:
        return False
    for row in data:
        if len(row) != 5:
            return False
        for column in row[:-1]:
            if type(column) != np.float64:
                return False
            
        if type(row[-1]) != str:
            return False
    
    return True

In [9]:
# read the file and convert the contents to the proper types and return list of lists
def read_file(): 
    data = [[] for x in range(ROWS)]                 # initialize empty list of lists length 150
    file = open('C:/Users/daven/COEN140/Lab4/test_dataset.txt', 'r')            # open file as read
    contents = [line.split() for line in file]      # put contents of file into list of lists of characters
    
    #iterate through contents stopping at each , in the list of characters to convert the first 4 elements into 
    #           np.float64 nd the last into a string to be loaded into data
    for row in range(len(contents)):                # iterate through contents
        i = 0
        for s in contents[row][0].split(","):       # usingn , as delimiter
            if i == (COLUMNS - 1):                              # at the last element just append as string
                data[row].append(s)
                break
            i += 1
            data[row].append(np.float64(s))         # append into data[row] the number converted to np.float64
    return data

In [10]:
# test the actual classifications compared to our predictions for QDA and LDA
# returns the percent error of predictions
def classification_compare(predicted, actuals):
    length = len(actuals)
    correct_predictions = 0
    for i in range(length):
        if predicted[i] == actuals[i]:
            correct_predictions += 1
        else:
            print("\nIncorrect prediction at index " + str(i))
            print("Predicted: " + predicted[i])
            print("Actual: ", actuals[i])
        
    return 100 - correct_predictions/length * 100

In [11]:
def category_means(category): 
    means = []
    for i in range(4): 
        temp_mean = 0
        for j in range(40):
            temp_mean += category[j][i]
        temp_mean = temp_mean/40
        means.append(temp_mean)
    return means
            

In [12]:
def variance(category, means):
    variances = []
    for i in range(4):
        new_arr = np.array([observation[i] for observation in category])
        variances.append(np.var(new_arr))
        
    return variances

In [13]:
# Given a matrix X of data representing all the different categories
#      a row vector of means for these categories means
#      and covariance matrix cov, compute probability using multivariate gaussian pdf.
def multivariate_gauss_pdf(X, means, cov):
    n = len(means)
    X = X-np.array(means).T
    return 1/((2*np.pi)**(n/2)*np.linalg.det(cov)**(0.5))*np.exp(-0.5*np.sum(X.dot(np.linalg.pinv(cov))*X,axis=1))

In [14]:
# Testing contentes of file converted successfully 
data = read_file()
print(test_dataset(data))

# split into training and testing lists 
training = data[:40:] + data[50:90:] + data[100:140:]
testing = data[40:50:] + data[90:100:] + data[140:150:]

# ensure proper length for training and testing items
print(len(training))
print(len(testing))

# using np array to better manipulate columns of data
training_array = np.array(training)
testing_array = np.array(testing)

True
120
30


In [15]:
# separation of X and y and testing/training
# splitting of arrays into arrays by classification of flower
# convert X from strings to floats using numpy
x_training = training_array[:,0:4].astype(np.float64)
x_testing = testing_array[:,0:4].astype(np.float64)
y_testing = testing_array[:,4]
y_training = training_array[:,4]

# splitting of arrays based on classification
setosa_training = x_training[:40]
versicolor_training = x_training[40:80]
virginica_training = x_training[80:120]

# check arrays were split correctly 
print("Arrays split sucessfully? " + 
      str(np.allclose(np.concatenate((setosa_training, np.concatenate((versicolor_training, virginica_training)))),x_training)))


Arrays split sucessfully? True


In [16]:
# computing means for each cateory
setosa_means = category_means(setosa_training)
versicolor_means = category_means(versicolor_training)
virginica_means = category_means(virginica_training)
# print(setosa_means)
# print(versicolor_means)
# print(virginica_means)

setosa_cov = np.cov(setosa_training, rowvar=False)
versicolor_cov = np.cov(versicolor_training, rowvar=False)
virginica_cov = np.cov(virginica_training, rowvar=False)
average_cov = (setosa_cov + versicolor_cov + virginica_cov) / 3
print(setosa_cov, end="\n\n")
print(versicolor_cov, end="\n\n")
print(virginica_cov, end="\n\n")

[[0.13112179 0.09897436 0.01298077 0.01362179]
 [0.09897436 0.13271795 0.00205128 0.0145641 ]
 [0.01298077 0.00205128 0.02958333 0.00458333]
 [0.01362179 0.0145641  0.00458333 0.00994231]]

[[0.27374359 0.08661538 0.17212821 0.05230769]
 [0.08661538 0.11087179 0.08087179 0.04538462]
 [0.17212821 0.08087179 0.20353205 0.07371795]
 [0.05230769 0.04538462 0.07371795 0.04307692]]

[[0.46794231 0.11041026 0.35777564 0.05125641]
 [0.11041026 0.11323077 0.08107692 0.04625641]
 [0.35777564 0.08107692 0.34532692 0.05930769]
 [0.05125641 0.04625641 0.05930769 0.07425641]]



In [17]:
# LDA classification for test data

#compute probabilities of each entry belonging to category of flower for testing group
set_prob_test = multivariate_gauss_pdf(x_testing, setosa_means, average_cov )
vers_prob_test = multivariate_gauss_pdf(x_testing, versicolor_means, average_cov )
virg_prob_test = multivariate_gauss_pdf(x_testing, virginica_means, average_cov )
print(set_prob_test, end="\n\n")
print(vers_prob_test, end="\n\n")
print(virg_prob_test, end="\n\n")



# classify based on comparisons of probabilities that each belongs to a certain category
lda_test_class = []
for i in range(30): 
    if set_prob_test[i] > vers_prob_test[i] and set_prob_test[i] > virg_prob_test[i]:
        lda_test_class.append('Iris-setosa')
    elif vers_prob_test[i] > set_prob_test[i] and vers_prob_test[i] > virg_prob_test[i]:
        lda_test_class.append('Iris-versicolor')
    elif virg_prob_test[i] > set_prob_test[i] and virg_prob_test[i] > vers_prob_test[i]:
        lda_test_class.append('Iris-virginica')
    
print(lda_test_class == y_testing)    ## should all be true
incorrect = (lda_test_class == y_testing).tolist().count(False)
print("LDA testing error: " + str(incorrect/(len(lda_test_class)) * 100) + "%")

[2.63344873e+00 7.04224498e-04 1.27971298e+00 4.45604179e-01
 5.59282137e-01 8.07876839e-01 9.57493988e-01 1.91769018e+00
 2.20760576e+00 3.21401964e+00 2.33083228e-22 5.96313501e-21
 3.38968253e-17 1.38160095e-14 6.31206388e-20 5.24117072e-17
 2.72283051e-18 1.78834668e-17 2.93847350e-12 5.88710827e-18
 4.52864493e-44 2.36542141e-37 1.50162818e-36 5.66795327e-44
 1.57319738e-45 2.56012607e-39 1.94491872e-35 1.27809885e-33
 9.26632376e-40 2.68549403e-32]

[1.26937594e-20 3.75317994e-14 4.40682304e-18 1.44453296e-15
 1.92864387e-17 4.16951788e-16 5.80686606e-22 1.23959537e-17
 2.36391665e-22 2.58473874e-19 3.89887158e-01 1.75564196e+00
 2.40223529e+00 1.90704099e-01 2.21969831e+00 5.70519926e-01
 1.80935567e+00 2.84554859e+00 2.23026776e-02 2.72580794e+00
 1.12026444e-06 5.62096625e-06 2.46006163e-03 4.69943472e-06
 1.27991079e-07 9.17879447e-06 3.70404092e-03 1.09290233e-02
 7.94671739e-06 1.36184511e-02]

[3.97231037e-39 2.02404342e-30 7.49657409e-36 1.81825261e-31
 6.66317141e-34 2.3

In [18]:
# QDA classification for test data
            
# compute probabilities of each entry belonging to category of flower for testing group
set_prob_test = multivariate_gauss_pdf(x_testing, setosa_means, setosa_cov)
vers_prob_test = multivariate_gauss_pdf(x_testing, versicolor_means, versicolor_cov)
virg_prob_test = multivariate_gauss_pdf(x_testing, virginica_means, virginica_cov)
print(set_prob_test, end="\n\n")
print(vers_prob_test, end="\n\n")
print(virg_prob_test, end="\n\n")



# classify based on comparisons of probabilities that each belongs to a certain category
qda_test_class = []
for i in range(30): 
    if set_prob_test[i] > vers_prob_test[i] and set_prob_test[i] > virg_prob_test[i]:
        qda_test_class.append('Iris-setosa')
    elif vers_prob_test[i] > set_prob_test[i] and vers_prob_test[i] > virg_prob_test[i]:
        qda_test_class.append('Iris-versicolor')
    elif virg_prob_test[i] > set_prob_test[i] and virg_prob_test[i] > vers_prob_test[i]:
        qda_test_class.append('Iris-virginica')
    
print(qda_test_class == y_testing)    ## should all be true
incorrect = (qda_test_class == y_testing).tolist().count(False)
print("QDA testing error: " + str(incorrect/(len(qda_test_class)) * 100) + "%")

[7.81177785e+000 1.81048629e-003 2.27890748e+000 5.59770776e-003
 1.00272575e-001 3.10718374e+000 3.02575674e+000 8.97886808e+000
 1.10596611e+001 1.61224185e+001 1.25995675e-075 2.35307034e-088
 6.17990203e-062 1.21308375e-037 1.94845153e-071 1.03616745e-065
 4.92882422e-070 4.70777807e-074 4.25477587e-034 2.42969159e-067
 2.20243455e-197 2.22033914e-167 8.96771811e-140 7.88258040e-203
 4.69068926e-209 4.04541604e-172 4.08462993e-138 6.88871015e-148
 2.77756948e-177 1.10142004e-130]

[5.88902311e-23 1.73532119e-12 1.70664995e-19 3.58520997e-18
 6.31867093e-21 2.01425700e-16 4.57657405e-26 5.79339655e-19
 1.12468803e-25 9.24900533e-21 2.40751868e-01 2.44508165e+00
 3.92526536e+00 3.67453462e-01 3.33068754e+00 4.88100376e-01
 2.71070381e+00 3.77762450e+00 9.19610454e-03 3.97986533e+00
 1.65374215e-08 1.09930077e-09 2.04944426e-03 1.00979403e-05
 2.91398336e-09 6.38674374e-09 2.35510388e-04 4.84677519e-03
 6.35435188e-06 7.32798211e-02]

[9.35547567e-36 4.79357840e-26 1.93466684e-29 6.64

In [19]:
# computing LDA classifications for the training data
# probabilites
set_prob_train = multivariate_gauss_pdf(x_training, setosa_means, average_cov)
vers_prob_train = multivariate_gauss_pdf(x_training, versicolor_means, average_cov)
virg_prob_train = multivariate_gauss_pdf(x_training, virginica_means, average_cov)

# classify based on comparisons of probabilities that each belongs to a certain category
lda_train_class = []
for i in range(120): 
    if set_prob_train[i] > vers_prob_train[i] and set_prob_train[i] > virg_prob_train[i]:
        lda_train_class.append('Iris-setosa')
    elif vers_prob_train[i] > set_prob_train[i] and vers_prob_train[i] > virg_prob_train[i]:
        lda_train_class.append('Iris-versicolor')
    elif virg_prob_train[i] > set_prob_train[i] and virg_prob_train[i] > vers_prob_train[i]:
        lda_train_class.append('Iris-virginica')
    
print(lda_train_class == y_training)    
incorrect = (lda_train_class == y_training).tolist().count(False)
error = incorrect/len(lda_train_class) * 100
print("LDA training error: " + str(error) + "%")

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True]
LDA training error: 2.5%


In [20]:
# computing QDA classifications for the training data
start = time.time()
# probabilites
set_prob_train = multivariate_gauss_pdf(x_training, setosa_means, setosa_cov)
vers_prob_train = multivariate_gauss_pdf(x_training, versicolor_means, versicolor_cov)
virg_prob_train = multivariate_gauss_pdf(x_training, virginica_means, virginica_cov)

# classify based on comparisons of probabilities that each belongs to a certain category
qda_train_class = []
for i in range(120): 
    if set_prob_train[i] > vers_prob_train[i] and set_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-setosa')
    elif vers_prob_train[i] > set_prob_train[i] and vers_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-versicolor')
    elif virg_prob_train[i] > set_prob_train[i] and virg_prob_train[i] > vers_prob_train[i]:
        qda_train_class.append('Iris-virginica')
    
print(qda_train_class == y_training)    
incorrect = (qda_train_class == y_training).tolist().count(False)
error = incorrect/len(qda_train_class) * 100
print("QDA training error: " + str(error) + "%")
end = time.time()
time_elapsed = end - start
print("Time elapsed for QDA analysis: " + str(time_elapsed * 1000) + " msec")

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]
QDA training error: 1.6666666666666667%
Time elapsed for QDA analysis: 2.563953399658203 msec


In [21]:
# 4. Is there any class linearly separable from other classes? Explain your answer based on 
# your experiments.

# computing LDA classifications for the training data

# probabilites
set_prob_train = multivariate_gauss_pdf(x_training, setosa_means, average_cov)
vers_prob_train = multivariate_gauss_pdf(x_training, versicolor_means, average_cov)
virg_prob_train = multivariate_gauss_pdf(x_training, virginica_means, average_cov)

# classify based on comparisons of probabilities that each belongs to a certain category
lda_train_class = []
for i in range(120): 
    if set_prob_train[i] > vers_prob_train[i] and set_prob_train[i] > virg_prob_train[i]:
        lda_train_class.append('Iris-setosa')
    elif vers_prob_train[i] > set_prob_train[i] and vers_prob_train[i] > virg_prob_train[i]:
        lda_train_class.append('Iris-versicolor')
    elif virg_prob_train[i] > set_prob_train[i] and virg_prob_train[i] > vers_prob_train[i]:
        lda_train_class.append('Iris-virginica')
    
prediction_results = (lda_train_class == y_training)
setosa_results = prediction_results[:40]
versicolor_results = prediction_results[40:80]
virginica_results = prediction_results[80:120]

setosa_incorrect = setosa_results.tolist().count(False)
versicolor_incorrect = versicolor_results.tolist().count(False)
virginica_incorrect = virginica_results.tolist().count(False)

setosa_error = setosa_incorrect/40 * 100
versicolor_error = versicolor_incorrect/40 * 100
virginica_error = virginica_incorrect/40 * 100

print("Setosa classification errors using LDA: " + str(setosa_error) + "%")
print("Versicolor classification errors using LDA: " + str(versicolor_error) + "%")
print("Virginica classification errors using LDA: " + str(virginica_error) + "%")

############################################ OBSERVATIONS #####################################################
# The only class linearly separable from other classes is the 'Iris-Setosa' class,
# as there were no classification errors when using LDA to classify the flower. 
# Versicolor and Virginica both had erros when classifying using LDA 

Setosa classification errors using LDA: 0.0%
Versicolor classification errors using LDA: 5.0%
Virginica classification errors using LDA: 2.5%


In [22]:
# 5. Assume the features are independent, i.e., ∑ is a diagonal matrix. Repeat Question 3, and 
# report your results. Also, please report the training time of this method and the original 
# QDA that you implemented in Question 3. 
 
# first create diagonal covariances
set_diag = np.zeros((4,4))
vers_diag = np.zeros((4,4))
virg_diag = np.zeros((4,4))
for row in range(setosa_cov.shape[0]): 
    for col in range(setosa_cov.shape[1]): 
        if row == col:
            set_diag[row][col] = setosa_cov[row][col]
            vers_diag[row][col] = versicolor_cov[row][col]
            virg_diag[row][col] = virginica_cov[row][col]
            
#QDA assuming independence (diagonal matrices)
start = time.time()
# probabilites
set_prob_train = multivariate_gauss_pdf(x_training, setosa_means, set_diag)
vers_prob_train = multivariate_gauss_pdf(x_training, versicolor_means, vers_diag)
virg_prob_train = multivariate_gauss_pdf(x_training, virginica_means, virg_diag)

# classify based on comparisons of probabilities that each belongs to a certain category
qda_train_class = []
for i in range(120): 
    if set_prob_train[i] > vers_prob_train[i] and set_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-setosa')
    elif vers_prob_train[i] > set_prob_train[i] and vers_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-versicolor')
    elif virg_prob_train[i] > set_prob_train[i] and virg_prob_train[i] > vers_prob_train[i]:
        qda_train_class.append('Iris-virginica')
    
print(qda_train_class == y_training)    
incorrect = (qda_train_class == y_training).tolist().count(False)
error = incorrect/len(qda_train_class) * 100
print("QDA training error: " + str(error) + "%")
end = time.time()
time_elapsed = end - start
print("Time elapsed for QDA analysis: " + str(time_elapsed * 1000) + " msec")

############################################# OBSERVATION #######################################################
# Although only marginally faster, with a large increase in data, the time difference would become more apparent,
# as the diagonal matrix did help with the efficiency of the QDA analysis. This could also have been more apparent
# had the calculations of means, covariances and other calculations done separate had been more concise. 
# The accuracy was still better than LDA with diagonal matrices and QDA. 

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True]
QDA training error: 4.166666666666666%
Time elapsed for QDA analysis: 1.6934871673583984 msec
